# Analyzing Mobile App Data

The purpose of this project is to analyze profitable apps to generate revenue for an imaginary app-building company. The revenue is ad-based and depends on user engagement from English-speaking audiences.

The data to be analyzed comes from the [Apple Store](https://www.kaggle.com/ramamet4/app-store-apple-data-set-10k-apps) and [Google Play store](https://dq-content.s3.amazonaws.com/350/googleplaystore.csv) in csv files that can be downloaded at the linked sources. The following code extracts data from these files to create list of list datasets.

In [1]:
def get_data(file):
    the_file = open(file)
    from csv import reader
    read_file = reader(the_file)
    apps_data = list(read_file)
    return apps_data

appledata = get_data('AppleStore.csv')
googledata = get_data('googleplaystore.csv')

This function returns specified rows of a data set, which can be reused to observe slices of data:

In [2]:
def explore_data(dataset, start, end, rows_and_columns=False):
    dataset_slice = dataset[start:end]    
    for row in dataset_slice:
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:
        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))

Column headers are removed and saved separately below to make the dataset easier to work with later.

In [3]:
explore_data(appledata, 0, 2) #16 columns
explore_data(googledata, 0, 2) #13 columns
appleheader = appledata[0]
del appledata[0]
googleheader = googledata[0]
del googledata[0]

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']




## Data cleaning

When a dataset has errors, duplicates or other unwanted data, it can be cleaned by removing the rows of problematic data. For example, it is known that one of the rows of Google App Store data is missing an entry, which shifts the columns. This is deleted below.

In [4]:
print(googleheader, "\n")

explore_data(googledata, 10471, 10472) #normal data
explore_data(googledata, 10472, 10473) #wrong data, missing Category

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver'] 

['Xposed Wi-Fi-Pwd', 'PERSONALIZATION', '3.5', '1042', '404k', '100,000+', 'Free', '0', 'Everyone', 'Personalization', 'August 5, 2014', '3.0.0', '4.0.3 and up']


['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']




In [5]:
del googledata[10472]

## Removing duplicates

Duplicates in the Google data are identified below. The set with the most reviews will be kept in favor of others so that newer reviews won't be left out.

In [6]:
googleduplicates = []
googleuniques = []

for app in googledata:
    name = app[0]
    if name not in googleuniques:
        googleuniques.append(name)
    else:
        googleduplicates.append(name)

In [7]:
#print(googleuniques)
print('Examples of duplicates', googleduplicates[0:3])
print('Expected Length: ', len(googleuniques))
print('Length duplicates: ', len(googleduplicates))
print('Expected Length: ', len(googledata)-1181)


Examples of duplicates ['Quick PDF Scanner + OCR FREE', 'Box', 'Google My Business']
Expected Length:  9659
Length duplicates:  1181
Expected Length:  9659


In [8]:
reviewsmax = {}

def get_names_max_reviews(dataset):
    for app in googledata:
        name = app[0]
        nreviews = float(app[3])
        if name in reviewsmax and nreviews>reviewsmax[name]:
            reviewsmax[name] = nreviews
        elif name not in reviewsmax:
            reviewsmax[name] = nreviews

get_names_max_reviews(googledata)
print('Length of apps with highest number of reviews: ', len(reviewsmax))

googleclean = []
alreadyadded = []

def clean_dataset_max_reviews(dataset):
    for app in googledata:
        name = app[0]
        nreviews = float(app[3])
        if nreviews==reviewsmax[name] and name not in alreadyadded:
            googleclean.append(app)
            alreadyadded.append(name)

clean_dataset_max_reviews(googledata)
print('Length cleaned data: ', len(alreadyadded))

Length of apps with highest number of reviews:  9659
Length cleaned data:  9659


## Isolating English-speaking apps
The Apple Store data doesn't have duplicates, but both sets have non-English-speaking apps, which are not useful for the goal of this analysis and will be removed below by filtering out ASCII characters higher than 27. ASCII character below 27 comprise English letters. Some apps contain a combination of emojis, which we want to include, but don't count a English. An imperfect, but simple way to include names with emojis is to exclude only apps with names having 3 or more non-English characters, which is done below.

In [9]:
def is_English(checkstring):
    countFilteredChars = 0
    for letter in checkstring:
        if ord(letter)>127:
            countFilteredChars+=1
        if countFilteredChars>=3:
            return False
    return True

print('Testing function:')
print(is_English('Instagram'))
print(is_English('爱奇艺PPS -《欢乐颂2》电视剧热播'))
print(is_English('Docs To Go™ Free Office Suite'))
print(is_English('Instachat 😜'))

def English_filter(dataset, header, namecol):
    newset = []
    index = header.index(namecol)
    for app in dataset:
        name = app[index]
        if is_English(name):
            newset.append(app)
    return newset
            
googleEnglish = English_filter(googleclean, googleheader, 'App')
print('Length google data - no duplicates English: ', len(googleEnglish))
appleEnglish = English_filter(appledata, appleheader, 'track_name') 
print('Length apple data - no duplicates English: ', len(appleEnglish))
#print(appleEnglish)
print(is_English('UC浏览器HD'))

Testing function:
True
False
True
True
Length google data - no duplicates English:  9597
Length apple data - no duplicates English:  6155
False


## Isolating free apps
We are only interested in free apps, so those are isolated here.

In [10]:
def get_free_apps(dataset, header, pricecol):
    priceindex = header.index(pricecol)
    freeapps = []    
    for app in dataset:
        price = (app[priceindex]).lower()
        if price == 'free' or price == '0' or price == '0.0':
            freeapps.append(app)
    return freeapps

freegoogle = get_free_apps(googleEnglish, googleheader, 'Price')
freeapple = get_free_apps(appleEnglish, appleheader, 'price')

print("Length free Google: ", len(freegoogle))
print("Length free Apple: ", len(freeapple))  

Length free Google:  8848
Length free Apple:  3203


## Further context and strategy
The app is planned to be made available on both the Apple Store and Google Play because our revenue is based on the number of users engaging with the app. Broader platform availability allows for the app to reach more people.

The strategy for validating and releasing the app is as follows:
1. Build a minimal Android version of the app, and add it to Google Play.
2. If the app has a good response from users, we develop it further.
3. If the app is profitable after six months, we build an iOS version of the app and add it to the App Store.

To be successful, we need to find app profiles that will be successful in both markets. This will require analysis of popular types of apps based on genre, ratings, and other metrics.

## Building frequency tables
The following functions build and display a frequency table for values in a given column of a dataset.

In [29]:
def freq_table(dataset, header, colname):
    index = header.index(colname)
    ftable = {}    
    for app in dataset:
        colval = app[index]
        if colval not in ftable:
            ftable[colval]=1
        else:
            ftable[colval]+=1
    sumf = sum(ftable.values()) #sum over values in frequency table
    for key in ftable:
        #ftable[key]="{:.2f}".format(ftable[key]/sumf*100) #show as a percentage formatted to 2 dec places
        ftable[key]=ftable[key]/sumf*100
    return ftable

#Allows for displaying sorted results:
def display_table(table):    
    #table = freq_table(dataset, header, colname)
    table_display = []
    for key in table:
        key_val_as_tuple = (table[key], key) #convert dictionary to tuple with value first, key second
        table_display.append(key_val_as_tuple)

    table_sorted = sorted(table_display, reverse = True)
    for entry in table_sorted:
        print(entry[1], ':', entry[0])
        
print('Freq table for apple prime_genre: \n')
applegenrefreq = freq_table(freeapple, appleheader, 'prime_genre')
display_table(applegenrefreq)

Freq table for apple prime_genre: 

Games : 58.25788323446769
Entertainment : 7.836403371838902
Photo & Video : 4.995316890415236
Education : 3.6840462066812365
Social Networking : 3.3093974399000934
Shopping : 2.5913206369029034
Utilities : 2.466437714642523
Sports : 2.1542304089915705
Music : 2.0605682172962845
Health & Fitness : 2.0293474867311896
Productivity : 1.7483609116453322
Lifestyle : 1.5610365282547611
News : 1.3424914142990947
Travel : 1.248829222603809
Finance : 1.0927255697783327
Weather : 0.8741804558226661
Food & Drink : 0.8117389946924758
Reference : 0.5307524196066188
Business : 0.5307524196066188
Book : 0.3746487667811427
Navigation : 0.18732438339057134
Medical : 0.18732438339057134
Catalogs : 0.1248829222603809


## Average apple ratings by genre

The above code can be used to find the average number of ratings per genre of a dataset. This will be used to help make a genre recommendation for an Apple Store app below.

In [ ]:
def avgnumratings(dataset, header, genrecolname, ratingsctcolname):
    freqtable = freq_table(dataset, header, genrecolname)
    dictionary={}
    for genre in freqtable:
        total = 0
        lengenre = 0
        genreindex = header.index(genrecolname)
        ratingsindex = header.index(ratingsctcolname)
        for app in dataset:           
            genreapp = app[genreindex]            
            if genreapp==genre:
                ratingct = float(app[ratingsindex])
                total+=ratingct
                lengenre+=1
        avgnumratings=total/lengenre
        dictionary[genre]=avgnumratings
    return dictionary

In [32]:
applenumratingsdict=avgnumratings(freeapple, appleheader, 'prime_genre', 'rating_count_tot')
display_table(applenumratingsdict)

Navigation : 86090.33333333333
Reference : 79350.4705882353
Social Networking : 71548.34905660378
Music : 57326.530303030304
Weather : 52279.892857142855
Book : 46384.916666666664
Food & Drink : 33333.92307692308
Finance : 32367.02857142857
Photo & Video : 28441.54375
Travel : 28243.8
Shopping : 27230.734939759037
Health & Fitness : 23298.015384615384
Sports : 23008.898550724636
Games : 22886.36709539121
News : 21248.023255813954
Productivity : 21028.410714285714
Utilities : 19156.493670886077
Lifestyle : 16815.48
Entertainment : 14195.358565737051
Business : 7491.117647058823
Education : 7003.983050847458
Catalogs : 4004.0
Medical : 612.0


## Recomendation for Apple Store app
It appears navigation apps have the highest average number of ratings. Genres with few single apps that have high numbers of ratings and few total apps may skew the results. Navigation is dominated by Waze and Google Maps, with very few other apps. However it appears genres lower down the list have many monoliths as well, such as Music and Social Networking with rating numbers in the millions for Pandora and Facebook. The app with the max number of ratings in reference is for the Bible, which is also relatively high, but to a lesser extent. I would recommend either **Social Networking** or doing a different kind of analysis to draw a recommendation for the Apple Store because using the average alone doesn't seem to be very useful with such a large spread. Many of the apps with the most use in non-social categories seem to have elements of social intent to them: Waze, Google Maps (reviews), Dictionary.com (communication), Google Translator, the Bible (religion). Also, Social Networking is a relatively much larger category, which can bring down the average while also signaling demand in that area.

In [33]:
def display_genre_dict(dataset, genrename, header, namecol, genrecol, ratingsctcol):
    nameindex=header.index(namecol)
    genreindex=header.index(genrecol)
    ratingsctindex=header.index(ratingsctcol)
    dictionary={}
    for app in dataset:
        name=app[nameindex]
        ratingsct=app[ratingsctindex]
        if app[genreindex]==genrename:
            dictionary[name]=float(ratingsct)
    display_dictionary(dictionary)

In [34]:
print('nav app counts')
display_genre_dict(freeapple, 'Navigation', appleheader, 'track_name', 'prime_genre', 'rating_count_tot')

nav app counts
Waze - GPS Navigation, Maps & Real-time Traffic : 345046.0
Google Maps - Navigation & Transit : 154911.0
Geocaching® : 12811.0
CoPilot GPS – Car Navigation & Offline Maps : 3582.0
ImmobilienScout24: Real Estate Search in Germany : 187.0
Railway Route Search : 5.0


In [18]:
print('reference app counts')
display_genre_dict(freeapple, 'Reference', appleheader, 'track_name', 'prime_genre', 'rating_count_tot')

reference app counts
Bible : 985920.0
Dictionary.com Dictionary & Thesaurus : 200047.0
Dictionary.com Dictionary & Thesaurus for iPad : 54175.0
Google Translate : 26786.0
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran : 18418.0
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition : 17588.0
Merriam-Webster Dictionary : 16849.0
Night Sky : 12122.0
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) : 8535.0
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools : 4693.0
GUNS MODS for Minecraft PC Edition - Mods Tools : 1497.0
Guides for Pokémon GO - Pokemon GO News and Cheats : 826.0
WWDC : 762.0
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free : 718.0
VPN Express : 14.0
Real Bike Traffic Rider Virtual Reality Glasses : 8.0
Jishokun-Japanese English Dictionary & Translator : 0.0


In [19]:
print('social app counts')
display_genre_dict(freeapple, 'Social Networking', appleheader, 'track_name', 'prime_genre', 'rating_count_tot')

social app counts
Facebook : 2974676.0
Pinterest : 1061624.0
Skype for iPhone : 373519.0
Messenger : 351466.0
Tumblr : 334293.0
WhatsApp Messenger : 287589.0
Kik : 260965.0
ooVoo – Free Video Call, Text and Voice : 177501.0
TextNow - Unlimited Text + Calls : 164963.0
Viber Messenger – Text & Call : 164249.0
Followers - Social Analytics For Instagram : 112778.0
MeetMe - Chat and Meet New People : 97072.0
We Heart It - Fashion, wallpapers, quotes, tattoos : 90414.0
InsTrack for Instagram - Analytics Plus More : 85535.0
Tango - Free Video Call, Voice and Chat : 75412.0
LinkedIn : 71856.0
Match™ - #1 Dating App. : 60659.0
Skype for iPad : 60163.0
POF - Best Dating App for Conversations : 52642.0
Timehop : 49510.0
Find My Family, Friends & iPhone - Life360 Locator : 43877.0
Whisper - Share, Express, Meet : 39819.0
Hangouts : 36404.0
LINE PLAY - Your Avatar World : 34677.0
WeChat : 34584.0
Badoo - Meet New People, Chat, Socialize. : 34428.0
Followers + for Instagram - Follower Analytics : 28

## Finding a recommendation for Google Play market
For Google Play, we will look at the number of installs per Category.

In [31]:
print ('Freq table for google category: \n')
freqgooglecategory = freq_table(freegoogle, googleheader, 'Category')
display_table(freqgooglecategory)

Freq table for google category: 

FAMILY : 18.942133815551536
GAME : 9.697106690777577
TOOLS : 8.453887884267631
BUSINESS : 4.599909584086799
PRODUCTIVITY : 3.899186256781193
LIFESTYLE : 3.887884267631103
FINANCE : 3.7070524412296564
MEDICAL : 3.5375226039783
SPORTS : 3.390596745027125
PERSONALIZATION : 3.322784810126582
COMMUNICATION : 3.2323688969258586
HEALTH_AND_FITNESS : 3.0854430379746836
PHOTOGRAPHY : 2.949819168173599
NEWS_AND_MAGAZINES : 2.802893309222423
SOCIAL : 2.667269439421338
TRAVEL_AND_LOCAL : 2.3395117540687163
SHOPPING : 2.2490958408679926
BOOKS_AND_REFERENCE : 2.1360759493670884
DATING : 1.8648282097649187
VIDEO_PLAYERS : 1.7970162748643763
MAPS_AND_NAVIGATION : 1.3901446654611211
FOOD_AND_DRINK : 1.2432188065099457
EDUCATION : 1.164104882459313
ENTERTAINMENT : 0.9606690777576853
LIBRARIES_AND_DEMO : 0.9380650994575045
AUTO_AND_VEHICLES : 0.9267631103074141
HOUSE_AND_HOME : 0.8024412296564195
WEATHER : 0.7911392405063291
EVENTS : 0.7120253164556962
PARENTING : 0.6555

In [41]:
def avginstalls(dataset, header, categcolname, installscolname):
    freqtable = freq_table(dataset, header, categcolname)
    dictionary={}
    for category in freqtable:
        total = 0
        lencateg = 0
        categindex = header.index(categcolname)
        installsindex = header.index(installscolname)
        for app in dataset:           
            categapp = app[categindex]            
            if categapp==category:
                rawinstalls = app[installsindex]
                cleaninstalls=rawinstalls.replace('+', '')
                cleaninstalls=cleaninstalls.replace(',', '')
                installs = float(cleaninstalls)
                total+=installs
                lencateg+=1
        avginstalls=total/lencateg
        dictionary[category]=avginstalls
    return dictionary

googleavginstalls = avginstalls(freegoogle, googleheader, 'Category', 'Installs')
print('Avg number of installs per category: \n')
display_table(googleavginstalls)

Avg number of installs per category: 

COMMUNICATION : 38590581.08741259
VIDEO_PLAYERS : 24727872.452830188
SOCIAL : 23253652.127118643
PHOTOGRAPHY : 17840110.40229885
PRODUCTIVITY : 16787331.344927534
GAME : 15544014.51048951
TRAVEL_AND_LOCAL : 13984077.710144928
ENTERTAINMENT : 11640705.88235294
TOOLS : 10830251.970588235
NEWS_AND_MAGAZINES : 9549178.467741935
BOOKS_AND_REFERENCE : 8814199.78835979
SHOPPING : 7036877.311557789
PERSONALIZATION : 5201482.6122448975
WEATHER : 5145550.285714285
HEALTH_AND_FITNESS : 4188821.9853479853
MAPS_AND_NAVIGATION : 4049274.6341463416
FAMILY : 3695641.8198090694
SPORTS : 3650602.276666667
ART_AND_DESIGN : 1986335.0877192982
FOOD_AND_DRINK : 1924897.7363636363
EDUCATION : 1833495.145631068
BUSINESS : 1712290.1474201474
LIFESTYLE : 1446158.2238372094
FINANCE : 1387692.475609756
HOUSE_AND_HOME : 1360598.042253521
DATING : 854028.8303030303
COMICS : 832613.8888888889
AUTO_AND_VEHICLES : 647317.8170731707
LIBRARIES_AND_DEMO : 638503.734939759
PARENTING 

## Google Play recommendation
The app category with the highest number of installs is Communication, and that is what I would recommend for the Google Play market. On the surface, there doesn't appear to be potential for much skewing of data in this case, and Communication is marginally different from another top category, Social.

In [45]:
def display_categ_dict(dataset, categname, header, namecol, categcol, installscol):
    nameindex=header.index(namecol)
    categindex=header.index(categcol)
    installsindex=header.index(installscol)
    dictionary={}
    for app in dataset:        
        if app[categindex]==categname:
            name=app[nameindex]
            rawinstalls = app[installsindex]
            cleaninstalls=rawinstalls.replace('+', '')
            cleaninstalls=cleaninstalls.replace(',', '')
            installs = float(cleaninstalls)
            dictionary[name]=installs
    display_dictionary(dictionary)

In [47]:
display_categ_dict(freegoogle, 'COMMUNICATION', googleheader, 'App', 'Category', 'Installs')

WhatsApp Messenger : 1000000000.0
Skype - free IM & video calls : 1000000000.0
Messenger – Text and Video Chat for Free : 1000000000.0
Hangouts : 1000000000.0
Google Chrome: Fast & Secure : 1000000000.0
Gmail : 1000000000.0
imo free video calls and chat : 500000000.0
Viber Messenger : 500000000.0
UC Browser - Fast Download Private & Secure : 500000000.0
LINE: Free Calls & Messages : 500000000.0
Google Duo - High Quality Video Calls : 500000000.0
imo beta free calls and text : 100000000.0
Yahoo Mail – Stay Organized : 100000000.0
Who : 100000000.0
WeChat : 100000000.0
UC Browser Mini -Tiny Fast Private & Secure : 100000000.0
Truecaller: Caller ID, SMS spam blocking & Dialer : 100000000.0
Telegram : 100000000.0
Opera Mini - fast web browser : 100000000.0
Opera Browser: Fast and Secure : 100000000.0
Messenger Lite: Free Calls & Messages : 100000000.0
Kik : 100000000.0
KakaoTalk: Free Calls & Text : 100000000.0
GO SMS Pro - Messenger, Free Themes, Emoji : 100000000.0
Firefox Browser fast &

In [50]:
display_categ_dict(freegoogle, 'VIDEO_PLAYERS', googleheader, 'App', 'Category', 'Installs')

YouTube : 1000000000.0
Google Play Movies & TV : 1000000000.0
MX Player : 500000000.0
VivaVideo - Video Editor & Photo Movie : 100000000.0
VideoShow-Video Editor, Video Maker, Beauty Camera : 100000000.0
VLC for Android : 100000000.0
Motorola Gallery : 100000000.0
Motorola FM Radio : 100000000.0
Dubsmash : 100000000.0
Vote for : 50000000.0
Vigo Video : 50000000.0
VMate : 50000000.0
Samsung Video Library : 50000000.0
Ringdroid : 50000000.0
MiniMovie - Free Video and Slideshow Editor : 50000000.0
LIKE – Magic Video Maker & Community : 50000000.0
KineMaster – Pro Video Editor : 50000000.0
HD Video Downloader : 2018 Best video mate : 50000000.0
DU Recorder – Screen Recorder, Video Editor, Live : 50000000.0
video player for android : 10000000.0
iMediaShare – Photos & Music : 10000000.0
YouTube Studio : 10000000.0
Video Player All Format : 10000000.0
Video Downloader - for Instagram Repost App : 10000000.0
Video Downloader : 10000000.0
Ustream : 10000000.0
Quik – Free Video Editor for photos

In [51]:
display_categ_dict(freegoogle, 'SOCIAL', googleheader, 'App', 'Category', 'Installs')

Instagram : 1000000000.0
Google+ : 1000000000.0
Facebook : 1000000000.0
Snapchat : 500000000.0
Facebook Lite : 500000000.0
VK : 100000000.0
Tumblr : 100000000.0
Tik Tok - including musical.ly : 100000000.0
Tango - Live Video Broadcast : 100000000.0
Pinterest : 100000000.0
LinkedIn : 100000000.0
Badoo - Free Chat & Dating App : 100000000.0
BIGO LIVE - Live Stream : 100000000.0
ooVoo Video Calls, Messaging & Stories : 50000000.0
Zello PTT Walkie Talkie : 50000000.0
SKOUT - Meet, Chat, Go Live : 50000000.0
POF Free Dating App : 50000000.0
MeetMe: Chat & Meet New People : 50000000.0
textPlus: Free Text & Calls : 10000000.0
magicApp Calling & Messaging : 10000000.0
YouNow: Live Stream Video Chat : 10000000.0
We Heart It : 10000000.0
Waplog - Free Chat, Dating App, Meet Singles : 10000000.0
TextNow - free text + calls : 10000000.0
Text free - Free Text + Call : 10000000.0
Text Me: Text Free, Call Free, Second Phone Number : 10000000.0
Tapatalk - 100,000+ Forums : 10000000.0
Tagged - Meet, Ch